In [212]:
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 9999)
pd.set_option('display.max_rows', 9999)

### Import Trades from file generated by code and create Prices dataframe

In [64]:
# Read generated buys
trades = pd.read_feather('/Users/renovieira/Desktop/trades.feather')

### Loop Through Days

In [264]:
# Min and Max Dates
min_date = trades['just_date'].min()
moving_date = min_date
max_date = trades['just_date'].max()# - timedelta(days=2)

# List of signals
list_of_signals = ['buy_signal?1', 'buy_signal?2','buy_signal?3','buy_signal?4']

# While moving date is less or equal to max_date
while moving_date <= max_date:
    
    # If it's the first date, the process is different, because holdings needs to be created yet
    if moving_date == min_date:
        # Figure out buys
        buys = extract_buys(trades, list_of_signals, moving_date)
        
        # Create holdings
        holdings_df = buys[['strategy', 'date', 'symbol', 'price', 'shares', 'total']]
        holdings_df.columns = ['strategy', 'buy_date', 'symbol', 'cost_per_unit', 'shares', 'cost_basis']
        
        # As of Date column
        holdings_df['as_of_date'] = moving_date
        holdings_df['day_close_price'] = 0
        
        # Symbols to string
        holdings_df['symbol'] = holdings_df['symbol'].astype(str)
        
        # Create RGL
        rgl_df = holdings_df.copy()
                
    else:
        
        # Prices
        prices = trades[trades['just_date'] == moving_date][['symbol', 'close_price_x']]
        prices.columns = ['symbol', 'day_close_price']
        
        # Previous holdings
        temp_holdings = holdings_df.drop('day_close_price', axis=1)
        previous_holdings = temp_holdings[temp_holdings['as_of_date'] == previous_day]
        previous_holdings.drop(['as_of_date'], axis=1, inplace=True)
        number_of_symbols = len(previous_holdings['symbol'].unique())
        
        # Figure out sells
        sells = extract_sells(trades, moving_date)
        sold_symbols = set(sells['symbol'])
        
        # Store Realized Gains/Losses in rgl_df
        if number_of_symbols > 1:
            
            today_rgl = pd.merge(previous_holdings, sells, how='inner', on='symbol')
            today_rgl['as_of_date'] = moving_date
            rgl_df = pd.concat([rgl_df, today_rgl])
        
        # Remove sells from previous day - Holdings
        previous_holdings = previous_holdings[~previous_holdings['symbol'].isin(sold_symbols)]
        
        # Figure out buys
        buys = extract_buys(trades, list_of_signals, moving_date)
        buys = buys[['strategy', 'date', 'symbol', 'price', 'shares', 'total']]
        buys.columns = ['strategy', 'buy_date', 'symbol', 'cost_per_unit', 'shares', 'cost_basis']
        
        # Today's holdings
        today = pd.concat([previous_holdings, buys])
        today['as_of_date'] = moving_date
        today = today.drop_duplicates()
        
        # Number of Symbols - buys
        number_of_symbols = len(today['symbol'].unique())
        
        # Prices
        if number_of_symbols > 1:
            today = pd.merge(today, prices, on='symbol', how='left')
            today['market_value'] = today['shares'] * today['day_close_price']
        
        # Append with holdings
        holdings_df = holdings_df.append(today)
        
        print(moving_date)
        
    previous_day = moving_date
    moving_date += timedelta(days=1)

2017-01-26
2017-01-27
2017-01-28
2017-01-29
2017-01-30
2017-01-31
2017-02-01
2017-02-02
2017-02-03
2017-02-04
2017-02-05
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-11
2017-02-12
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-18
2017-02-19
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-25
2017-02-26
2017-02-27
2017-02-28
2017-03-01
2017-03-02
2017-03-03
2017-03-04
2017-03-05
2017-03-06
2017-03-07
2017-03-08
2017-03-09
2017-03-10
2017-03-11
2017-03-12
2017-03-13
2017-03-14
2017-03-15
2017-03-16
2017-03-17
2017-03-18
2017-03-19
2017-03-20
2017-03-21
2017-03-22
2017-03-23
2017-03-24
2017-03-25
2017-03-26
2017-03-27
2017-03-28
2017-03-29
2017-03-30
2017-03-31
2017-04-01
2017-04-02
2017-04-03
2017-04-04
2017-04-05
2017-04-06
2017-04-07
2017-04-08
2017-04-09
2017-04-10
2017-04-11
2017-04-12
2017-04-13
2017-04-14
2017-04-15
2017-04-16
2017-04-17
2017-04-18
2017-04-19
2017-04-20
2017-04-21
2017-04-22
2017-04-23
2017-04-24
2017-04-25
2017-04-26

2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01
2019-04-02
2019-04-03
2019-04-04
2019-04-05
2019-04-06
2019-04-07
2019-04-08
2019-04-09
2019-04-10
2019-04-11
2019-04-12
2019-04-13
2019-04-14
2019-04-15
2019-04-16
2019-04-17
2019-04-18
2019-04-19
2019-04-20
2019-04-21
2019-04-22
2019-04-23
2019-04-24
2019-04-25
2019-04-26
2019-04-27
2019-04-28
2019-04-29
2019-04-30
2019-05-01
2019-05-02
2019-05-03
2019-05-04
2019-05-05
2019-05-06
2019-05-07
2019-05-08
2019-05-09
2019-05-10
2019-05-11

In [275]:
rgl_df.dropna(subset=['market_value']).groupby('strategy').sum()['diff']

strategy
buy_signal?1   -32738.176311
buy_signal?2   -32413.511998
buy_signal?3   -24747.447505
buy_signal?4   -26835.662509
Name: diff, dtype: float64

In [278]:
sells

,symbol,sell_date,sold_price,order_type
8714,KOMP,2020-05-08,33.430000,sell
11198,VOOG,2020-05-08,173.190002,sell
12854,CHAU,2020-05-08,20.250000,sell
23888,JOE,2020-05-08,18.150000,sell
31593,VOX,2020-05-08,89.489998,sell
32421,BBN,2020-05-08,23.129999,sell
35351,FICO,2020-05-08,371.100006,sell
44577,HMI,2020-05-08,13.500000,sell
46065,SUB,2020-05-08,106.879997,sell
47721,BEP,2020-05-08,49.799999,sell


### Create DataFrames of Buys and Sells
#### Buys

In [244]:
def extract_buys(df, list_of_signals, buy_date):
    
    # Create empty list to store results
    buys_list = []
    
    # Columns
    columns_list = ['symbol', 'date', 'price', 'order_type', 'shares', 'total', 'strategy']
    
    # Empty dataframe
    empty_df = pd.DataFrame([[0,0,0,0,0,0]], columns = ['symbol', 'date', 'price', 'order_type', 'shares', 'total'])
    
    # Loop through different strategies
    for buy_signal in list_of_signals:

        # Create dataframe with buys
        buys = trades[(trades['just_date'] == buy_date) 
                     & (trades[buy_signal] == True)
                    & (trades['sell_signal?'] != True)][['symbol', 'just_date', 'close_price_x']]

        # Create some other columns
        buys['order_type'] = 'buy'
        buys['shares'] = 100 / buys['close_price_x']
        buys['total'] = buys['shares'] * buys['close_price_x']
        buys['strategy'] = buy_signal
       
        # If length is zero then create empty record
        if len(buys) == 0:
            buys_none = empty_df
            buys_none['strategy'] = buy_signal
            buys_list.append(buys_none.values.tolist()[0])
            
        else:
            buys.columns = columns_list
            buys_list.append(buys.values.tolist()[0])
    
    buys_df = pd.DataFrame(buys_list, columns = columns_list)
    return buys_df

### Sells

In [178]:
def extract_sells(trades, date):
    sells = trades[(trades['just_date'] == date) 
                & (trades['sell_signal?'] == True)][['symbol', 'just_date', 'close_price_x']]
    sells['order_type'] = 'sell'
    sells.columns = ['symbol', 'sell_date', 'sold_price', 'order_type']
    
    return sells

In [58]:
buys_list = []

# Loop through different strategies
for buy_signal in ['buy_signal?1', 'buy_signal?2','buy_signal?3','buy_signal?4']:
    
    # Create dataframe with buys
    buys = trades[(trades['just_date'] == max_date) 
                 & (trades[buy_signal] == True)
                & (trades['sell_signal?'] != True)][['symbol', 'just_date', 'close_price_x']]
    
    # Create some other columns
    buys['strategy'] = buy_signal
    buys['order_type'] = 'buy'
    buys['shares'] = 1
    buys['total'] = buys['shares'] * buys['close_price_x']
    
    # Append to list
    buys_list.append(buys)
    
# Concat all dataframes
buys  = pd.concat(buys_list)
buys.columns = ['symbol', 'date', 'price', 'strategy', 'order_type', 'shares', 'total']
buys.head()

,symbol,date,price,strategy,order_type,shares,total
333,RCEL,2020-05-08,5.980000,buy_signal?1,buy,1,5.980000
366,RETA,2020-05-08,162.229996,buy_signal?1,buy,1,162.229996
732,EHTH,2020-05-08,116.180000,buy_signal?1,buy,1,116.180000
1143,GMRE,2020-05-08,10.510000,buy_signal?1,buy,1,10.510000
1737,DGAZ,2020-05-08,217.339996,buy_signal?1,buy,1,217.339996


#### Sells

In [62]:
sells = trades[(trades['just_date'] == max_date) 
                & (trades['sell_signal?'] == True)][['symbol', 'just_date', 'close_price_x']]
sells['order_type'] = 'sell'
sells.columns = ['symbol', 'sell_date', 'price', 'order_type']
sells.head()

,symbol,sell_date,price,order_type
36,KOMP,2020-05-08,33.430000,sell
45,VOOG,2020-05-08,173.190002,sell
51,CHAU,2020-05-08,20.250000,sell
93,JOE,2020-05-08,18.150000,sell
123,VOX,2020-05-08,89.489998,sell


### Holdings

In [59]:
# Create Holdings DataFrame
holdings_df = buys[['strategy', 'date', 'symbol', 'price', 'shares', 'total']]
holdings_df.columns = ['strategy', 'buy_date', 'symbol', 'cost_per_unit', 'shares', 'cost_basis']
holdings_df

,strategy,buy_date,symbol,cost_per_unit,shares,cost_basis
316,buy_signal?1,2020-05-06,COO,294.959991,1,294.959991
334,buy_signal?1,2020-05-06,ERY,64.489998,1,64.489998
520,buy_signal?1,2020-05-06,BZQ,32.099998,1,32.099998
772,buy_signal?1,2020-05-06,PFNX,5.740000,1,5.740000
982,buy_signal?1,2020-05-06,EPZM,17.920000,1,17.920000
2779,buy_signal?1,2020-05-06,DRIP,8.030000,1,8.030000
2830,buy_signal?1,2020-05-06,WU,19.469999,1,19.469999
316,buy_signal?2,2020-05-06,COO,294.959991,1,294.959991
772,buy_signal?2,2020-05-06,PFNX,5.740000,1,5.740000
1813,buy_signal?2,2020-05-06,BMRN,95.000000,1,95.000000


In [63]:
pd.merge(holdings_df, sells, how='left', on='symbol')

,strategy,buy_date,symbol,cost_per_unit,shares,cost_basis,sell_date,price,order_type
0,buy_signal?1,2020-05-06,COO,294.959991,1,294.959991,2020-05-08,304.519989,sell
1,buy_signal?1,2020-05-06,ERY,64.489998,1,64.489998,NaN,NaN,NaN
2,buy_signal?1,2020-05-06,BZQ,32.099998,1,32.099998,NaN,NaN,NaN
3,buy_signal?1,2020-05-06,PFNX,5.740000,1,5.740000,NaN,NaN,NaN
4,buy_signal?1,2020-05-06,EPZM,17.920000,1,17.920000,NaN,NaN,NaN
5,buy_signal?1,2020-05-06,DRIP,8.030000,1,8.030000,NaN,NaN,NaN
6,buy_signal?1,2020-05-06,WU,19.469999,1,19.469999,2020-05-08,19.930000,sell
7,buy_signal?2,2020-05-06,COO,294.959991,1,294.959991,2020-05-08,304.519989,sell
8,buy_signal?2,2020-05-06,PFNX,5.740000,1,5.740000,NaN,NaN,NaN
9,buy_signal?2,2020-05-06,BMRN,95.000000,1,95.000000,NaN,NaN,NaN
